In [1]:
# Purpose: Interpolate the monthly-average CAM5 kernels to daily resolution.

# By: Ty Janoski
# Updated: 06.08.21

## Interpolate Pendergrass CAM5 kernels from monthly -> daily

In [2]:
# import statments
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import nc_time_axis
import scipy as sp
from cftime import DatetimeNoLeap
import scipy.io as io

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [3]:
# we need to add a normalization factor to the q kernels
norm = io.loadmat('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/CAM5_qkernel_normfactor.mat')['dlogqdt']
norm = norm.transpose([3,2,1,0])

In [4]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on january of a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [5]:
# make the midpoints
midpoints = [DatetimeNoLeap(1,1,15,12),DatetimeNoLeap(1,2,14,0),
            DatetimeNoLeap(1,3,15,12),DatetimeNoLeap(1,4,15,0),
            DatetimeNoLeap(1,5,15,12),DatetimeNoLeap(1,6,15,0),
            DatetimeNoLeap(1,7,15,12),DatetimeNoLeap(1,8,15,12),
            DatetimeNoLeap(1,9,15,0),DatetimeNoLeap(1,10,15,12),
            DatetimeNoLeap(1,11,15,0),DatetimeNoLeap(1,12,15,12)]
# make new dates 
dates = xr.cftime_range(start="0001-01-01 12:00:00",end="0001-12-31 12:00:00",freq='D',calendar='noleap')
# file endings
files = ['alb.kernel','PS','q.kernel','t.kernel','ts.kernel']
for f in files:
    print(f)
    # read in kernels
    kern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/kernels/'+f+'.nc',use_cftime=True)
    
    # overwrite time axis
    kern['time'] = midpoints
    
    # we need to include a normalization factor in the q kernel
    if(f=='q.kernel'):
        kern['FLNS'] = kern.FLNS/norm
        kern['FLNSC'] = kern.FLNSC/norm
        kern['FLNT'] = kern.FLNT/norm
        kern['FLNTC'] = kern.FLNTC/norm
        kern['FSNS'] = kern.FSNS/norm
        kern['FSNSC'] = kern.FSNSC/norm
        kern['FSNT'] = kern.FSNT/norm
        kern['FSNTC'] = kern.FSNTC/norm
    
    # slight issue: we want the daily kernels to be periodic (so that there is no discontinuity between Dec 31st and Jan 1st)
    # we will have to "pad" with a preceding decemeber and succeeding January
    pre = kern.isel(time=11)
    pre['time'] = DatetimeNoLeap(0,12,15,12)

    suc = kern.isel(time=0)
    suc['time'] = DatetimeNoLeap(2,1,15,12)
    
    kern_extend = xr.concat([pre,kern,suc],dim='time')
    
    # interpolate (linear)
    out = kern_extend.interp(time=dates,method='linear')
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/'+f+'.nc'
    
    # save out
    out.to_netcdf(path)

alb.kernel
PS
q.kernel
t.kernel
ts.kernel


## We are going to repeat the process for the CAM4 and CAM5 stratosphere-adjusted forcings.

### CAM4 first

In [3]:
# make the midpoints
midpoints = [DatetimeNoLeap(2,1,15,12),DatetimeNoLeap(2,2,14,0),
            DatetimeNoLeap(2,3,15,12),DatetimeNoLeap(2,4,15,0),
            DatetimeNoLeap(2,5,15,12),DatetimeNoLeap(2,6,15,0),
            DatetimeNoLeap(2,7,15,12),DatetimeNoLeap(2,8,15,12),
            DatetimeNoLeap(2,9,15,0),DatetimeNoLeap(2,10,15,12),
            DatetimeNoLeap(2,11,15,0),DatetimeNoLeap(2,12,15,12)]

# make new dates 
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0002-12-31 12:00:00",freq='D',calendar='noleap')

mon_avgs = []
# read in CAM4 files
for i in range(1,13,1):
    # add leading 0
    month = str(i).zfill(2)
    
    # read in file
    file_in = xr.open_dataset('/dx01/chiodo/data4/port/f1850.cam.port.4xCO2.adj.001/cam4.4xco2.forcing.'+month+'.nc')
    
    # overwrite time axis on file
    file_in['time'] = midpoints[i-1]
    
    # append to list
    mon_avgs.append(file_in)
    
# as before, we need to add on leading and trailing month for boundary conditions in interpolation
pre = mon_avgs[-1].copy()
pre['time'] = DatetimeNoLeap(1,12,15,12)
suc = mon_avgs[0].copy()
suc['time'] = DatetimeNoLeap(3,1,15,12)
mon_avgs.insert(0,pre)
mon_avgs.append(suc)

# turn list to dataset of montly averages
monthly = xr.concat(mon_avgs,dim='time')

# interpolate
CAM4_daily = monthly.interp(time=dates,method='linear')
path = '/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc'

CAM4_daily.to_netcdf(path)

### CAM5 RRTMG

In [7]:
# make the midpoints
midpoints = [DatetimeNoLeap(2,1,15,12),DatetimeNoLeap(2,2,14,0),
            DatetimeNoLeap(2,3,15,12),DatetimeNoLeap(2,4,15,0),
            DatetimeNoLeap(2,5,15,12),DatetimeNoLeap(2,6,15,0),
            DatetimeNoLeap(2,7,15,12),DatetimeNoLeap(2,8,15,12),
            DatetimeNoLeap(2,9,15,0),DatetimeNoLeap(2,10,15,12),
            DatetimeNoLeap(2,11,15,0),DatetimeNoLeap(2,12,15,12)]

# make new dates 
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0002-12-31 12:00:00",freq='D',calendar='noleap')

mon_avgs = []
# read in CAM4 files
for i in range(1,13,1):
    # add leading 0
    month = str(i).zfill(2)
    
    # read in file
    file_in = xr.open_dataset('/dx01/chiodo/data4/port/PCCAM5.e122.beta.f19_f19.4xCO2.001/cam5.4xco2.forcing.'+month+'.nc')
    
    # overwrite time axis on file
    file_in['time'] = midpoints[i-1]
    
    # append to list
    mon_avgs.append(file_in)
    
# as before, we need to add on leading and trailing month for boundary conditions in interpolation
pre = mon_avgs[-1].copy()
pre['time'] = DatetimeNoLeap(1,12,15,12)
suc = mon_avgs[0].copy()
suc['time'] = DatetimeNoLeap(3,1,15,12)
mon_avgs.insert(0,pre)
mon_avgs.append(suc)

# # turn list to dataset of montly averages
monthly = xr.concat(mon_avgs,dim='time')

# # interpolate
CAM5_daily = monthly.interp(time=dates,method='linear')
path = '/dx02/janoski/cesm-LE/strat_adj_rf/CAM5_RRTMG_strat_adj_RF.nc'

CAM5_daily.to_netcdf(path)